In [112]:
import sys
sys.path.append("/mnt/nvme2tb/ffp/code/mlfires/ML_fires_al/")
import pathlib
import pandas as pd
import xarray as xr
import numpy as np
import os
import crop_dataset
import geopandas as gpd

'''
Adds a hash string ID in the dataset from x y coords 
'''
def applyid(df):
    df['xposst'] = (df['x'] * 10000).apply('{:06.0f}'.format)
    df['yposst'] = (df['y'] * 10000).apply('{:06.0f}'.format)
    df['id'] = df['xposst'] + df['yposst']
    df.drop(columns=['xposst', 'yposst'], inplace=True)
    return df

'''
Creates coarsen dataset subgrid from the initial dataset.


Input 
    coarsen_c : The number of points in the subgrid is 1/( coarsen_c * coarsen_c) 
                We need to use odd numbers to maintain existing points coordinates in the subgrid.
    file : The csv file to merge

Output  
    Corsen tabular pandas dataframe that contains only the instances of the points in the grid
'''

def getcoarsedf(coarsen_c, dfday):
    ds = xr.open_dataset('/mnt/nvme2tb/ffp/datasets/images/20211030_df.nc')
    dsc = ds.coarsen(y=coarsen_c, boundary='trim').mean().coarsen(x=coarsen_c, boundary='trim').mean()
    coordtupar = dsc.stack(dim=['x', 'y']).dim.to_numpy()
    coordnp = np.array([*coordtupar])
    dfcoocrds = pd.DataFrame(coordnp, columns=['x', 'y'], dtype=float)
    dfcoocrds = applyid(dfcoocrds)
    coarsen_df = pd.merge(dfday, dfcoocrds, on=['id'], suffixes=("", "_c")).drop(columns=['x_c', 'y_c'])
    return coarsen_df

def extract_day(date):
    csvfolder = '/mnt/nvme2tb/ffp/datasets/prod/'
    '''
    gdfperif = crop_dataset.getperif()
    crop_dataset.cropfile(os.path.join(csvfolder,date,'%s_norm.csv'%date),
                      os.path.join(csvfolder,date, gdfperif, '_greece'),
                      usexyid='id')
    '''

    csvfile = os.path.join(csvfolder, date, '%s_norm_greece.csv' % date)

    # csv for xai input
    
    coarsen_coef=31
    dfday = pd.read_csv(csvfile, dtype={'id': str})
    coarsedf = getcoarsedf(coarsen_coef, dfday)
    
    dfpred=extract_xy(pd.read_csv(os.path.join(csvfolder,date,"%s_pred_greece.csv"%date), dtype={'id': str}))
    
    dfexids=getextrapoints(coarsedf, dfpred, coarsen_coef)
    
    coarsedfext = pd.merge(dfexids, dfday, on=['id'])
  
    xaifolder = '/mnt/nvme2tb/ffp/datasets/xai/%s' % date
   
    if not os.path.isdir(xaifolder): os.makedirs(xaifolder)
    csvcoarse = os.path.join(xaifolder, '%s_xai_inp.csv' % date)
    #coarsedf.to_csv(csvcoarse, index=False)
    return coarsedfext

def extract_xy(dfxai):
    dfxai['x']=dfxai['id'].str.slice(0,6).astype(int)/10000
    dfxai['y']=dfxai['id'].str.slice(6,12).astype(int)/10000
    return dfxai

def getcenters(dfcoarse, dfpred):
    dfcenter=pd.merge(dfcoarse[['id','max_temp']], dfpred, on='id', how='right')
    dfcenter.loc[~dfcenter['max_temp'].isna(), 'max_temp']=1
    dfcenter.loc[dfcenter['max_temp'].isna(), 'max_temp']=0
    dfcenter.rename(columns={'max_temp':'center'},inplace=True)
    dfcenter['center']=dfcenter['center'].astype(int)
    
    geom = gpd.points_from_xy(dfcenter['x'], dfcenter['y'], crs=4326)
    gdfcenter = gpd.GeoDataFrame(dfcenter, geometry=geom)
    #gdfcenter.set_crs(4326)
    #gdfcenter=gdfcenter.to_crs(2100)
    #gdfcenter['x']=gdfcenter['geometry'].x
    #gdfcenter['y']=gdfcenter['geometry'].y
    #dfcenter=gdfcenter.drop(columns='geometry')
      
    #dfcenteri=dfcenter.set_index(['x', 'y'])
    #dscenter=dfcenteri.to_xarray()
    
    return dfcenter

def getextrapoints(dfcoarse, dfpred, coarsen_coef):
    #merge the points from the coarsening with the rest of the dataset. 
    #Create "center" column to mark which points are the chosen after the coarsening
    dfcenter=pd.merge(dfcoarse[['id','max_temp']], dfpred, on='id', how='right')
    dfcenter.loc[~dfcenter['max_temp'].isna(), 'max_temp']=1
    dfcenter.loc[dfcenter['max_temp'].isna(), 'max_temp']=0
    dfcenter.rename(columns={'max_temp':'center'},inplace=True)
    dfcenter['center']=dfcenter['center'].astype(int)
    
    #create geodataframe with point geometries 
    geom = gpd.points_from_xy(dfcenter['x'], dfcenter['y'], crs=4326)
    gdfcenter = gpd.GeoDataFrame(dfcenter, geometry=geom)
    gdfcenter=gdfcenter.to_crs(2100)
    
    # spatial join of center points with all points around centers 
    # using the coarsen coefficient to calculate a square buffer
    gdfcenter2=gdfcenter.loc[gdfcenter['center']==1].copy()
    gdfcenter2['geometry']=gdfcenter2.geometry.buffer((coarsen_coef-1)/2*500,cap_style=3)
    gdfcenter2.drop(columns=['ypred0','ypred1','x','y'], inplace=True)     
    gdfsjoin=gdfcenter2.sjoin(gdfcenter, how="left")
    
    
    # find and select one point id for each risk level
    # for the points in the buffer keeping the initial center point
    sampleids=[]
    for ind in gdfcenter2.index:
        sampleids+=[gdfcenter2.loc[gdfcenter2.index==ind,'id'].item()]
        for risk in range(1,6):
            allrows=gdfsjoin.loc[(gdfsjoin.index==ind)\
                         &(gdfsjoin['id_left']!=gdfsjoin['id_right'])\
                         &(gdfsjoin['risk_left']!=gdfsjoin['risk_right'])\
                         &(gdfsjoin['risk_right']==risk)]
            if not allrows.empty:
                #print(ind,allrows.iloc[0]["id_right"])
                sampleids+=[allrows.iloc[0]["id_right"]]
    dfextids = pd.DataFrame(sampleids, columns=['id'])
    return dfextids
    

In [ ]:
for d in range(25,29):
    date='202308'+str(d)
    extract_day(date)

In [18]:
dfpred=extract_xy(pd.read_csv("/mnt/nvme2tb/ffp/datasets/prod/20230825/20230825_pred_greece.csv", dtype={'id': str}))

In [107]:
pd.merge(pd.DataFrame(sampleids, columns=['id']), dfpred, on=['id'])

,id,ypred0,ypred1,x,y,risk
0,198017397712,0.981292,0.018708,19.8017,39.7712,1
1,198120397042,0.865274,0.134726,19.8120,39.7042,2
2,198017397042,0.693812,0.306188,19.8017,39.7042,3
3,197811397042,0.302181,0.697819,19.7811,39.7042,4
4,197708397042,0.050401,0.949599,19.7708,39.7042,5
...,...,...,...,...,...,...
2368,268323376940,0.039367,0.960633,26.8323,37.6940,5
2369,268787376786,0.956592,0.043408,26.8787,37.6786,1
2370,268014376425,0.883034,0.116966,26.8014,37.6425,2
2371,267962376425,0.574963,0.425037,26.7962,37.6425,3


In [19]:
dfcoarse=extract_day('20230825')

In [113]:
extract_day('20230825')

,id,x,y,dom_dir,dom_vel,res_max,dir_max,max_temp,min_temp,mean_temp,...,corine_gr4,corine_gr5,corine_gr21,corine_gr22,corine_gr23,corine_gr24,corine_gr31,corine_gr32,corine_gr33,fire
0,198017397712,0.050504,0.702992,1.0,0.061934,0.030523,1.0,0.775728,0.876531,0.824093,...,0.0,0.000000,0.0,1.000000,0.0,0.000000,0.0,0.000000,0.00000,0
1,198120397042,0.051502,0.693554,1.0,0.046182,0.087866,8.0,0.817915,0.875582,0.844581,...,0.0,0.000000,0.0,0.995999,0.0,0.000000,0.0,0.004001,0.00000,0
2,198017397042,0.050504,0.693554,2.0,0.039812,0.091597,8.0,0.811170,0.869235,0.837448,...,0.0,0.000000,0.0,0.022497,0.0,0.976744,0.0,0.000000,0.00000,0
3,197811397042,0.048508,0.693554,3.0,0.047485,0.085940,8.0,0.802685,0.861531,0.831930,...,0.0,0.000000,0.0,0.401607,0.0,0.026639,0.0,0.571754,0.00000,0
4,197708397042,0.047510,0.693554,3.0,0.047485,0.085940,8.0,0.802685,0.861531,0.831930,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.472870,0.52713,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2368,268323376940,0.731105,0.410425,1.0,0.234696,0.205059,1.0,0.849920,0.941994,0.902045,...,0.0,0.000000,0.0,0.000000,0.0,0.518187,0.0,0.481813,0.00000,0
2369,268787376786,0.735596,0.408247,1.0,0.157282,0.126851,1.0,0.874667,0.968725,0.924265,...,0.0,0.000000,0.0,1.000000,0.0,0.000000,0.0,0.000000,0.00000,0
2370,268014376425,0.728111,0.403165,7.0,0.185421,0.155278,7.0,0.840597,0.902685,0.874488,...,0.0,0.095154,0.0,0.904846,0.0,0.000000,0.0,0.000000,0.00000,0
2371,267962376425,0.727613,0.403165,7.0,0.185421,0.155278,7.0,0.840597,0.902685,0.874488,...,0.0,0.483842,0.0,0.516158,0.0,0.000000,0.0,0.000000,0.00000,0


In [24]:
dfcenter, dscenter, gdfcenter=getcenters(dfcoarse, dfpred)
gdfcenter

,id,center,ypred0,ypred1,x,y,risk,geometry
0,195234397712,0,0.513199,0.486801,116383.008556,4.411670e+06,3,POINT (116383.009 4411669.910)
1,195285397661,0,0.466710,0.533290,116791.708505,4.411082e+06,3,POINT (116791.709 4411081.689)
2,195749398949,0,0.843961,0.156039,121477.025768,4.425187e+06,2,POINT (121477.026 4425186.742)
3,195749398898,0,0.748692,0.251308,121448.918297,4.424620e+06,2,POINT (121448.918 4424620.397)
4,195749398846,0,0.965407,0.034593,121420.262792,4.424043e+06,1,POINT (121420.263 4424042.948)
...,...,...,...,...,...,...,...,...
500496,270488377198,0,0.432870,0.567130,768575.662076,4.178815e+06,3,POINT (768575.662 4178815.172)
500497,270540377198,0,0.681825,0.318175,769034.106807,4.178830e+06,3,POINT (769034.107 4178830.123)
500498,270540377146,0,0.780087,0.219913,769052.943147,4.178253e+06,2,POINT (769052.943 4178252.989)
500499,270591377198,0,0.607320,0.392680,769483.735837,4.178845e+06,3,POINT (769483.736 4178844.811)


In [16]:
geom = gpd.points_from_xy(dfcenter['x'], dfcenter['y'], crs=4326)
gdfcenter = gpd.GeoDataFrame(dfcenter, geometry=geom)
gdfcenter=gdfcenter.to_crs(2100)
gdfcenter
#gdfcenter['x']=gdfcenter['geometry'].x
#    gdfcenter['y']=gdfcenter['geometry'].y
#    dfcenter=gdfcenter.drop(columns='geometry')

,id,center,ypred0,ypred1,x,y,risk,geometry
0,195234397712,0,0.513199,0.486801,116383.008556,4.411670e+06,3,POINT (inf inf)
1,195285397661,0,0.466710,0.533290,116791.708505,4.411082e+06,3,POINT (inf inf)
2,195749398949,0,0.843961,0.156039,121477.025768,4.425187e+06,2,POINT (inf inf)
3,195749398898,0,0.748692,0.251308,121448.918297,4.424620e+06,2,POINT (inf inf)
4,195749398846,0,0.965407,0.034593,121420.262792,4.424043e+06,1,POINT (inf inf)
...,...,...,...,...,...,...,...,...
500496,270488377198,0,0.432870,0.567130,768575.662076,4.178815e+06,3,POINT (inf inf)
500497,270540377198,0,0.681825,0.318175,769034.106807,4.178830e+06,3,POINT (inf inf)
500498,270540377146,0,0.780087,0.219913,769052.943147,4.178253e+06,2,POINT (inf inf)
500499,270591377198,0,0.607320,0.392680,769483.735837,4.178845e+06,3,POINT (inf inf)


In [25]:
gdfcenter['x']=gdfcenter['geometry'].x
gdfcenter['y']=gdfcenter['geometry'].y
#gdfcenter['x']=gdfcenter['x'].astype(int)
#gdfcenter['y']=gdfcenter['y'].astype(int)
gdfcenter
#    dfcenter=gdfcenter.drop(columns='geometry')

,id,center,ypred0,ypred1,x,y,risk,geometry
0,195234397712,0,0.513199,0.486801,116383.008556,4.411670e+06,3,POINT (116383.009 4411669.910)
1,195285397661,0,0.466710,0.533290,116791.708505,4.411082e+06,3,POINT (116791.709 4411081.689)
2,195749398949,0,0.843961,0.156039,121477.025768,4.425187e+06,2,POINT (121477.026 4425186.742)
3,195749398898,0,0.748692,0.251308,121448.918297,4.424620e+06,2,POINT (121448.918 4424620.397)
4,195749398846,0,0.965407,0.034593,121420.262792,4.424043e+06,1,POINT (121420.263 4424042.948)
...,...,...,...,...,...,...,...,...
500496,270488377198,0,0.432870,0.567130,768575.662076,4.178815e+06,3,POINT (768575.662 4178815.172)
500497,270540377198,0,0.681825,0.318175,769034.106807,4.178830e+06,3,POINT (769034.107 4178830.123)
500498,270540377146,0,0.780087,0.219913,769052.943147,4.178253e+06,2,POINT (769052.943 4178252.989)
500499,270591377198,0,0.607320,0.392680,769483.735837,4.178845e+06,3,POINT (769483.736 4178844.811)


In [40]:
gdfcenter2=gdfcenter.loc[gdfcenter['center']==1].copy()

In [41]:
gdfcenter2['geometry']=gdfcenter2.geometry.buffer(15*500,cap_style=3)

In [42]:
gdfcenter2.drop(columns=['ypred0','ypred1','x','y'], inplace=True)

In [43]:
gdfsjoin=gdfcenter2.sjoin(gdfcenter, how="left")

In [44]:
gdfsjoin

,id_left,center_left,risk_left,geometry,index_right,id_right,center_right,ypred0,ypred1,x,y,risk_right
720,198017397712,1,1,"POLYGON ((147730.346 4418012.844, 147730.346 4...",809,198120397042,0,0.865274,0.134726,140764.672537,4.403032e+06,2
720,198017397712,1,1,"POLYGON ((147730.346 4418012.844, 147730.346 4...",771,198069397042,0,0.905078,0.094922,140327.246195,4.403052e+06,2
720,198017397712,1,1,"POLYGON ((147730.346 4418012.844, 147730.346 4...",733,198017397042,0,0.693812,0.306188,139881.242325,4.403073e+06,3
720,198017397712,1,1,"POLYGON ((147730.346 4418012.844, 147730.346 4...",696,197966397042,0,0.884346,0.115654,139443.814923,4.403094e+06,2
720,198017397712,1,1,"POLYGON ((147730.346 4418012.844, 147730.346 4...",660,197914397042,0,0.717586,0.282414,138997.809972,4.403115e+06,3
...,...,...,...,...,...,...,...,...,...,...,...,...
499820,268323376940,1,5,"POLYGON ((757075.614 4182852.050, 757075.614 4...",499807,268323377610,0,0.875822,0.124178,749350.522582,4.182788e+06,2
499820,268323376940,1,5,"POLYGON ((757075.614 4182852.050, 757075.614 4...",499839,268375377610,0,0.849805,0.150195,749808.690019,4.182802e+06,2
499820,268323376940,1,5,"POLYGON ((757075.614 4182852.050, 757075.614 4...",499871,268426377610,0,0.788180,0.211820,750258.047047,4.182815e+06,2
499820,268323376940,1,5,"POLYGON ((757075.614 4182852.050, 757075.614 4...",499903,268478377610,0,0.832482,0.167518,750716.215511,4.182829e+06,2


In [103]:
sampleids=[]
for ind in gdfcenter2.index:
    sampleids+=[gdfcenter2.loc[gdfcenter2.index==ind,'id'].item()]
    for risk in range(1,6):
        allrows=gdfsjoin.loc[(gdfsjoin.index==ind)\
                     &(gdfsjoin['id_left']!=gdfsjoin['id_right'])\
                     &(gdfsjoin['risk_left']!=gdfsjoin['risk_right'])\
                     &(gdfsjoin['risk_right']==risk)]
        if not allrows.empty:
            #print(ind,allrows.iloc[0]["id_right"])
            sampleids+=[allrows.iloc[0]["id_right"]]


In [105]:
len(sampleids)

2373

In [82]:
ind=720
risk=2
for risk in range(1,6):
    allrows=gdfsjoin.loc[(gdfsjoin.index==ind)\
                     &(gdfsjoin['id_left']!=gdfsjoin['id_right'])\
                     &(gdfsjoin['risk_left']!=gdfsjoin['risk_right'])\
                     &(gdfsjoin['risk_right']==risk)]
    print(allrows)
    if not allrows.empty:
            print(ind,allrows.iloc[0]["risk_right"])

Empty GeoDataFrame
Columns: [id_left, center_left, risk_left, geometry, index_right, id_right, center_right, ypred0, ypred1, x, y, risk_right]
Index: []
Empty GeoDataFrame
Columns: [id_left, center_left, risk_left, geometry, index_right, id_right, center_right, ypred0, ypred1, x, y, risk_right]
Index: []


In [77]:
allrows.iloc[222]

id_left                                              198017397712
center_left                                                     1
risk_left                                                       1
geometry        POLYGON ((147730.34646358632 4418012.843719457...
index_right                                                   996
id_right                                             198378398073
center_right                                                    0
ypred0                                                   0.827113
ypred1                                                   0.172887
x                                                   143510.293075
y                                                  4414376.771758
risk_right                                                      2
Name: 720, dtype: object

In [32]:
center_coords = dscenter.where(dscenter['center'] == 1, drop=True)

In [33]:
center_coords

<xarray.Dataset>
Dimensions:  (x: 47, y: 41)
Coordinates:
  * x        (x) float64 19.8 19.96 20.28 20.44 20.6 ... 27.15 27.79 27.95 28.11
  * y        (y) float64 34.82 34.98 35.14 35.3 ... 41.21 41.37 41.53 41.69
Data variables:
    id       (x, y) object nan nan nan nan nan nan ... nan nan nan nan nan nan
    center   (x, y) float64 nan nan nan nan nan nan ... nan nan nan nan nan nan
    ypred0   (x, y) float64 nan nan nan nan nan nan ... nan nan nan nan nan nan
    ypred1   (x, y) float64 nan nan nan nan nan nan ... nan nan nan nan nan nan
    risk     (x, y) float64 nan nan nan nan nan nan ... nan nan nan nan nan nan

In [34]:
for coord_name, coord_values in center_coords.coords.items():
    print(f"Coordinate Name: {coord_name}")
    print(f"Coordinate Values:\n{coord_values}")
    print("------")

Coordinate Name: x
Coordinate Values:
<xarray.DataArray 'x' (x: 47)>
array([19.8017, 19.9615, 20.2811, 20.4409, 20.6006, 20.7604, 20.9202, 21.08  ,
       21.2398, 21.3996, 21.5594, 21.7191, 21.8789, 22.0387, 22.1985, 22.3583,
       22.5181, 22.6779, 22.8377, 22.9974, 23.1572, 23.317 , 23.4768, 23.6366,
       23.7964, 23.9562, 24.1159, 24.2757, 24.4355, 24.5953, 24.7551, 24.9149,
       25.0747, 25.2345, 25.3942, 25.554 , 25.7138, 25.8736, 26.0334, 26.1932,
       26.353 , 26.5127, 26.8323, 27.1519, 27.791 , 27.9508, 28.1106])
Coordinates:
  * x        (x) float64 19.8 19.96 20.28 20.44 20.6 ... 27.15 27.79 27.95 28.11
------
Coordinate Name: y
Coordinate Values:
<xarray.DataArray 'y' (y: 41)>
array([34.8179, 34.9776, 35.1374, 35.2972, 35.457 , 35.9364, 36.0962, 36.2559,
       36.5755, 36.7353, 36.8951, 37.0549, 37.2147, 37.3744, 37.5342, 37.694 ,
       37.8538, 38.0136, 38.1734, 38.3332, 38.4929, 38.6527, 38.8125, 38.9723,
       39.1321, 39.2919, 39.4517, 39.6115, 39.7712, 39.931

In [53]:
nearest_coords = dscenter.sel( y=39.7712, x=19.8017, method='nearest', tolerance=1)

In [54]:
nearest_coords

<xarray.Dataset>
Dimensions:  ()
Coordinates:
    x        float64 19.8
    y        float64 39.77
Data variables:
    id       object '198017397712'
    center   float64 1.0
    ypred0   float64 0.9813
    ypred1   float64 0.01871
    risk     float64 1.0

In [39]:
dscenter.sel(x=slice(19.5,19.7), y=slice(39.7,39.9))

<xarray.Dataset>
Dimensions:  (x: 15, y: 38)
Coordinates:
  * x        (x) float64 19.52 19.53 19.57 19.58 ... 19.68 19.69 19.69 19.7
  * y        (y) float64 39.7 39.71 39.71 39.72 ... 39.88 39.88 39.89 39.89
Data variables:
    id       (x, y) object nan nan nan nan nan nan ... nan nan nan nan nan nan
    center   (x, y) float64 nan nan nan nan nan nan ... nan nan nan nan nan nan
    ypred0   (x, y) float64 nan nan nan nan nan nan ... nan nan nan nan nan nan
    ypred1   (x, y) float64 nan nan nan nan nan nan ... nan nan nan nan nan nan
    risk     (x, y) float64 nan nan nan nan nan nan ... nan nan nan nan nan nan

In [50]:
for xc in dscenter.coords['x']:
    print(xc.item())

19.5234
19.5285
19.5749
19.5801
19.5852
19.6522
19.6574
19.6625
19.6677
19.6728
19.678
19.6832
19.6883
19.6935
19.6986
19.7038
19.7089
19.7141
19.7192
19.7244
19.7295
19.7347
19.7399
19.745
19.7502
19.7553
19.7605
19.7656
19.7708
19.7759
19.7811
19.7862
19.7914
19.7966
19.8017
19.8069
19.812
19.8172
19.8223
19.8275
19.8326
19.8378
19.8429
19.8481
19.8533
19.8584
19.8636
19.8687
19.8739
19.879
19.8842
19.8893
19.8945
19.8996
19.9048
19.9099
19.9151
19.9203
19.9254
19.9306
19.9357
19.9409
19.946
19.9512
19.9563
19.9615
19.9666
19.9718
19.977
19.9821
19.9873
19.9924
19.9976
20.0027
20.0079
20.013
20.0182
20.0233
20.0285
20.0337
20.0388
20.044
20.0491
20.0543
20.0594
20.0646
20.0697
20.0749
20.08
20.0852
20.0904
20.0955
20.1007
20.1058
20.111
20.1213
20.1316
20.1367
20.1419
20.1471
20.1522
20.1574
20.1625
20.1677
20.1728
20.178
20.1831
20.1883
20.1934
20.1986
20.2038
20.2089
20.2141
20.2192
20.2244
20.2295
20.2347
20.2398
20.245
20.2501
20.2553
20.2604
20.2656
20.2708
20.2759
20.2811
20.28

In [45]:
dscenter.coords["x"]

<xarray.DataArray 'x' (x: 1642)>
array([19.5234, 19.5285, 19.5749, ..., 28.2292, 28.2343, 28.2395])
Coordinates:
  * x        (x) float64 19.52 19.53 19.57 19.58 ... 28.22 28.23 28.23 28.24